In [ ]:
#pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357252 sha256=5e5003e1c6455f954a855ef713f8753966d15886e2865af79a8b0d461b771330
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#url_md="https://drive.google.com/file/d/1XwjHueThE58wygOjBQTXlTXkeUbsKMWL/view?usp=drive_link"
md= pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
md.shape

(45466, 24)

In [ ]:
links_small = pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/links_small.csv')
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [ ]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
md=md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')

In [ ]:
small_md = md[md['id'].isin(links_small)]
small_md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40224,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,2016-07-29,77000000.0,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0
40503,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",...,2016-09-15,0.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0
44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0
44826,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0


In [ ]:
small_md['tagline'] = small_md['tagline'].fillna('')
small_md['description'] = small_md['overview'] + small_md['tagline']
small_md['description'] = small_md['description'].fillna('')

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(small_md['description'])
tfidf_matrix

<9099x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 540591 stored elements in Compressed Sparse Row format>

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [ ]:
small_md = small_md.reset_index()
titles = small_md['title']
indices = pd.Series(small_md.index, index=small_md['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [ ]:
get_recommendations('Forrest Gump').head(10)

8435                           Frozen
3764                      Serendipity
353                     Jason's Lyric
655                        Phenomenon
7980                    Perfect Sense
7856                      Project Nim
4425                      City of God
3318    Long Night's Journey Into Day
5040                        Jane Eyre
5439                     The Notebook
Name: title, dtype: object

In [ ]:
get_recommendations('The Shawshank Redemption').head(10)

8206            Les Misérables
505                    Chasers
5650             Sherlock, Jr.
5690           The Blue Dahlia
5983              The Woodsman
4051                  Breakout
2729            Coogan's Bluff
408                The Getaway
7409                 A Prophet
6193    The 40 Year Old Virgin
Name: title, dtype: object

In [ ]:
# Metadata-based recommendations
credits = pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/credits.csv')
keywords = pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/keywords.csv')

In [ ]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [ ]:
md.shape

(45463, 24)

In [ ]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [ ]:
small_md=md[md['id'].isin(links_small)]
small_md.shape

(9219, 27)

In [ ]:
small_md['cast'] = small_md['cast'].apply(literal_eval)
small_md['crew'] = small_md['crew'].apply(literal_eval)
small_md['keywords'] = small_md['keywords'].apply(literal_eval)
small_md['cast_size'] = small_md['cast'].apply(lambda x: len(x))
small_md['crew_size'] = small_md['crew'].apply(lambda x: len(x))

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
small_md['director'] = small_md['crew'].apply(get_director)

In [ ]:
small_md['cast'] = small_md['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
small_md['cast'] = small_md['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
small_md['keywords'] = small_md['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
small_md['cast'] = small_md['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
small_md['director'] = small_md['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
small_md['director'] = small_md['director'].apply(lambda x: [x,x, x])

In [ ]:
s = small_md.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [ ]:
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

In [ ]:
s = s[s > 1]

In [ ]:
stemmer = SnowballStemmer('english')
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words


In [ ]:
small_md['keywords'] = small_md['keywords'].apply(filter_keywords)
small_md['keywords'] = small_md['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
small_md['keywords'] = small_md['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
small_md['soup'] = small_md['keywords'].apply(lambda x: ' '.join(x)) + ' ' + \
                   small_md['cast'].apply(lambda x: ' '.join(x)) + ' ' + \
                   small_md['director'].apply(lambda x: ' '.join(x)) + ' ' + \
                   small_md['genres'].apply(lambda x: ' '.join(x))
small_md['soup']

0        jealousi toy boy friendship friend rivalri boy...
1        boardgam disappear basedonchildren'sbook newho...
2        fish bestfriend duringcreditssting waltermatth...
3        basedonnovel interracialrelationship singlemot...
4        babi midlifecrisi confid age daughter motherda...
                               ...                        
40952    friendship sidneypoitier wendycrewson jayo.san...
41172    bollywood akshaykumar ileanad'cruz eshagupta t...
41225    bollywood hrithikroshan poojahegde kabirbedi a...
41391    monster godzilla giantmonst destruct kaiju hir...
41669    music documentari paulmccartney ringostarr joh...
Name: soup, Length: 9219, dtype: object

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(small_md['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [ ]:
small_md = small_md.reset_index()
titles = small_md['title']
indices = pd.Series(small_md.index, index=small_md['title'])

In [ ]:
get_recommendations('The Shawshank Redemption').head(10)

3900           The Majestic
2547         The Green Mile
6900               The Mist
2545       Cradle Will Rock
34         Dead Man Walking
947             Bob Roberts
6156        We're No Angels
252     Murder in the First
7117            Dead Leaves
4548              Dark Blue
Name: title, dtype: object

In [ ]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
7648                     Inception
2085                     Following
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [ ]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
m = vote_counts.quantile(0.95)
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = small_md.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,wr
7648,Inception,14075,8,7.919065
8613,Interstellar,11187,8,7.898936
6623,The Prestige,4510,8,7.762198
3381,Memento,4168,8,7.744491
8031,The Dark Knight Rises,9263,7,6.922734
6218,Batman Begins,7511,7,6.905676
8872,Captain America: Civil War,7462,7,6.905090
8419,Man of Steel,6462,6,5.953020
8467,Kick-Ass 2,2275,6,5.880165
9024,Batman v Superman: Dawn of Justice,7189,5,5.013324


In [ ]:
improved_recommendations('The Shawshank Redemption')

,title,vote_count,vote_average,wr
2547,The Green Mile,4166,8,7.744379
1348,Witness,390,7,6.081529
2618,Malcolm X,377,7,6.066641
34,Dead Man Walking,350,7,6.034124
4258,The Experiment,319,7,5.993879
6900,The Mist,1438,6,5.826326
1296,Air Force One,859,6,5.748011
6208,The Longest Yard,804,6,5.736734
8890,True Story,536,6,5.663315
7545,District 13: Ultimatum,323,6,5.567441


In [ ]:
# Collaborative Filtering_Using SVM
reader = Reader()
ratings = pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/ratings_small.csv')
ratings.head()




,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
from surprise.model_selection import train_test_split
from surprise import accuracy

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)



In [ ]:
svd = SVD()
svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)



RMSE: 0.9071


0.9071247663939462

In [ ]:
from surprise.model_selection import cross_validate, KFold

# Define k-fold cross-validator
kf = KFold(n_splits=5)

# Perform cross-validation
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=kf, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8986  0.9021  0.8993  0.8942  0.8868  0.8962  0.0053  
MAE (testset)     0.6909  0.6943  0.6944  0.6889  0.6811  0.6899  0.0049  
Fit time          3.97    4.11    1.50    1.45    1.49    2.51    1.25    
Test time         0.59    0.41    0.13    0.11    0.11    0.27    0.20    


{'test_rmse': array([0.89859714, 0.90209435, 0.89934166, 0.89419302, 0.88682048]),
 'test_mae': array([0.69086916, 0.69433983, 0.69442538, 0.68892456, 0.68110755]),
 'fit_time': (3.9703283309936523,
  4.108587026596069,
  1.5022130012512207,
  1.454902172088623,
  1.4897997379302979),
 'test_time': (0.5858967304229736,
  0.4126749038696289,
  0.126878023147583,
  0.1118619441986084,
  0.11350488662719727)}

In [ ]:
svd.predict(1, 1061, 3)

Prediction(uid=1, iid=1061, r_ui=3, est=2.9532147454964237, details={'was_impossible': False})

In [ ]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


# **HYBRID ENGINE** for Personalised User Recommendation


In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = pd.read_csv('/content/drive/My Drive//Movie Recommender_datasets/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(small_md[['title', 'id']], on='id').set_index('title')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = small_md.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,280,3.520096
4987,Battle Royale,992.0,7.3,3176,3.473122
1011,The Terminator,4208.0,7.4,218,3.304625
907,A Christmas Carol,34.0,7.0,25842,3.301872
344,True Lies,1138.0,6.8,36955,3.119815
922,The Abyss,822.0,7.1,2756,3.116893
974,Aliens,3282.0,7.7,679,3.081285
987,Alien,4564.0,7.9,348,3.075129
899,Platoon,1236.0,7.5,792,3.051371
8401,Star Trek Into Darkness,4479.0,7.4,54138,2.987521


In [ ]:
hybrid(200, 'Avatar')

,title,vote_count,vote_average,id,est
974,Aliens,3282.0,7.7,679,3.817652
899,Platoon,1236.0,7.5,792,3.781317
2014,Fantastic Planet,140.0,7.6,16306,3.703112
922,The Abyss,822.0,7.1,2756,3.559558
987,Alien,4564.0,7.9,348,3.547148
522,Terminator 2: Judgment Day,4274.0,7.7,280,3.540818
344,True Lies,1138.0,6.8,36955,3.528646
1011,The Terminator,4208.0,7.4,218,3.504526
907,A Christmas Carol,34.0,7.0,25842,3.414356
7646,Predators,1231.0,6.0,34851,3.379078


In [ ]:
hybrid(1, 'Forrest Gump')

,title,vote_count,vote_average,id,est
1041,Back to the Future,6239.0,8.0,105,3.259151
413,Heaven & Earth,51.0,6.4,31642,3.195176
8255,Silver Linings Playbook,4840.0,7.0,82693,3.108263
2414,Who Framed Roger Rabbit,1466.0,7.2,856,3.100962
7455,A Christmas Carol,1137.0,6.6,17979,3.076977
3470,Uncommon Valor,54.0,6.2,9967,3.068347
3945,Little Dieter Needs to Fly,25.0,7.8,57976,3.038611
8256,Flight,2449.0,6.5,87502,3.036511
1274,Contact,1338.0,7.2,686,2.978597
9031,The Walk,1151.0,6.9,285783,2.944059


In [ ]:
hybrid(500, 'Forrest Gump')

,title,vote_count,vote_average,id,est
1041,Back to the Future,6239.0,8.0,105,3.723182
2368,Return with Honor,2.0,4.0,56235,3.380103
9031,The Walk,1151.0,6.9,285783,3.310160
1933,Romancing the Stone,477.0,6.6,9326,3.294375
1274,Contact,1338.0,7.2,686,3.246499
3239,Cast Away,3304.0,7.5,8358,3.217112
2414,Who Framed Roger Rabbit,1466.0,7.2,856,3.164779
3945,Little Dieter Needs to Fly,25.0,7.8,57976,3.153760
7455,A Christmas Carol,1137.0,6.6,17979,3.149832
8159,Ted,4811.0,6.3,72105,3.108754
